##**Importing the modules**

In [0]:
import librosa
import librosa.display
import numpy as np
import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import regularizers
from sklearn.utils import shuffle
import pandas as pd
import glob
import matplotlib.pyplot as plt
import scipy.io.wavfile
import sys
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.model_selection import train_test_split
import os
import json
from google.colab import drive

## Loading dataset from Google Drive

In [0]:
drive.mount('/gdrive')
!cp "/gdrive/My Drive/aud.zip" "/aud.zip"
!unzip /aud.zip -d /
!clear
!sudo mkdir /data
!cp /aud/* /data/

## Setting path for  dataset

In [0]:
path = '/data/'  #path here
mylist= os.listdir(path)
type(mylist)
mylist[:5]

## Setting labels to data points

In [0]:
feeling_list=[]
for item in mylist:
    if item[1:3]=='Ne':
        feeling_list.append(1)
    elif item[1:3]=='Sa':
        feeling_list.append(2)
    elif item[1:3]=='Su':
        feeling_list.append(3)
    elif item[1:3]=='Fe':
        feeling_list.append(4)
    elif item[1:3]=='An':
        feeling_list.append(5)
    elif item[1:3]=='Di':
        feeling_list.append(6)
    elif item[1:3]=='Ha':
        feeling_list.append(7)

labels = pd.DataFrame(feeling_list)
labels[-3:]

## Extraction of features using libROSA

In [0]:
df = pd.DataFrame(columns=['feature'])
bookmark=0
for y in mylist:
  X, sample_rate = librosa.load('/data/'+ y, res_type='kaiser_fast',duration=None ,sr=None,offset=0.0)
  sample_rate = np.array(sample_rate)
  mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=18), axis=0)
  feature = mfccs
  #rms=np.mean(librosa.feature.rmse(y=None, S=sample_rate, frame_length=1024, hop_length=256, center=True, pad_mode='reflect'),axis=0)  #extra feature
  #feature1=rms
  #ffeature= np.concatenate((feature, feature1), axis=None)  #adding up different features to dataframe
  df.loc[bookmark] =[feature]
  
  bookmark=bookmark+1

## Processing the dataframe

In [0]:
df[:]
df3 = pd.DataFrame(df['feature'].values.tolist())
test=df3
newdf = pd.concat([df3,labels], axis=1)
rnewdf = newdf.rename(index=str, columns={"0": "label"})
rnewdf = shuffle(newdf)

In [0]:
rnewdf[:]

In [0]:
rnewdf=rnewdf.fillna(0)
rnewdf.size

253936

## Setting dimesions for LSTM and encoding the labels

In [0]:
train = rnewdf

trainfeatures = train.iloc[:, :-1]
trainlabel = test.iloc[:,-2:]


X_train = np.array(trainfeatures)
y_train = np.array(feeling_list)

y_train = y_train.reshape((538,1))

lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))

YY=y_train
XX =np.expand_dims(X_train, axis=2)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Checking input shapes

In [0]:
XX.shape

(538, 471, 1)

In [0]:
YY.shape

(538, 7)

## Splitting the data points for training and testing

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(XX,YY, test_size = 0.1)

## Model creation

In [0]:
trainX=XX
trainy=YY
#trainX= X_train
#trainy= Y_train
#testX= X_test
#testy= Y_test

n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
model = Sequential()
model.add(LSTM(230, input_shape=(None, n_features)))
model.add(Dropout(0.1))
model.add(Dense(230, activation='relu'))
model.add(Dense(230, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

## Optimizers and Compilation

In [0]:
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
#opt = keras.optimizers.Adam(lr=0.001, beta_1=0.99, beta_2=0.99, epsilon=None, decay=0, amsgrad=False)
#opt = keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=1e-6, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])    

## Training and Epochs

In [0]:
verbose = 1
epochs = 250
model.fit(trainX, trainy, epochs=epochs, verbose=verbose)
#score, accuracy = model.evaluate(testX, testy, verbose=verbose)
#print("Test Score: ",score,"   Test Accuracy: ",accuracy)
#print(accuracy)    


# **Saving the model as JSON and weights as HD5**

In [0]:
model_name = 'mk1_final.h5'
save_dir = os.path.join(os.getcwd(), '/saved_models')
# Save model and weights
if not os.path.isdir(save_dir):
  os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

model_json = model.to_json()
with open("mk1_final.json", "w") as json_file:
  json_file.write(model_json)

## **Loading up the model and weights**

In [0]:
# loading json and creating model
from keras.models import model_from_json
json_file = open('mk1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/saved_models/mk1.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

## **Testing model on test set**

In [0]:
"""## Predicting emotions on the test data"""

preds = loaded_model.predict(X_test, 
                         batch_size=32, 
                         verbose=1)

preds

preds1=preds.argmax(axis=1)

preds1

abc = preds1.astype(int).flatten()

predictions = (lb.inverse_transform((abc)))

preddf = pd.DataFrame({'predictedvalues': predictions})
preddf[:10]

actual=Y_test.argmax(axis=1)
abc123 = actual.astype(int).flatten()
actualvalues = (lb.inverse_transform((abc123)))

actualdf = pd.DataFrame({'actualvalues': actualvalues})
actualdf[:10]

finaldf = actualdf.join(preddf)

"""## Actual v/s Predicted emotions"""
finaldf[:]

# **Testing on external input**

In [0]:
import os
import pandas as pd
import librosa
import glob 

#livedf= pd.DataFrame(columns=['feature'])
X, sample_rate = librosa.load('/4Su1.WAV', res_type='kaiser_fast',duration=None,sr=22050,offset=0.0)
sample_rate = np.array(sample_rate)
mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
featurelive = mfccs
livedf2 = featurelive
livedf2.shape

In [0]:
livedf2= pd.DataFrame(data=livedf2)

livedf2 = livedf2.stack().to_frame().T

x,y = livedf2.shape

twodim= np.expand_dims(livedf2, axis=2)

livepreds = loaded_model.predict(twodim)


livepreds

livepreds1=livepreds.argmax(axis=1)

liveabc = livepreds1.astype(int).flatten()

livepredictions = (lb.inverse_transform((liveabc)))
livepredictions
